iPSC: P9_01 and MN: M6_03 are samples that were sequenced to over 100 million reads each. I need to downsample them, in intervals of 5 million reads, maybe 5-10 iterations each. I'll use [`seqtk`](https://github.com/lh3/seqtk) for the downsampling.

After I downsample, then I'll need to create a manifest file and submit these data for the analysis pipeline - probably minus everything after mapping.

In [5]:
cd /home/obotvinnik/projects/downsample_singlecell/data

/oasis/tscc/scratch/obotvinnik/projects/downsample_singlecell/data


## SOme files are improperly paired

In [8]:
! ls -lha P9_01_97000000reads_iteration1_R*

-rw-r--r-- 1 obotvinnik yeo-group   20 Feb 15 06:12 P9_01_97000000reads_iteration1_R1.fastq.gz
-rw-r--r-- 1 obotvinnik yeo-group 371M Feb 15 06:25 P9_01_97000000reads_iteration1_R2.fastq.gz


In [11]:
! grep P9_01_97000000reads /home/obotvinnik/projects/singlecell_pnms/data/downsample*

In [12]:
import glob
import os
import pandas as pd

/home/obotvinnik/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:810: UserWarning: Found matplotlib configuration in ~/.matplotlib/. To conform with the XDG base directory standard, this configuration location has been deprecated on Linux, and the new location is now /home/obotvinnik/.config/matplotlib/. Please move your configuration there to ensure that matplotlib will continue to find it in the future.
  _get_xdg_config_dir())


In [15]:
reads = pd.DataFrame()
# reads['properly_paired'] = np.nan


for read1 in glob.iglob('*R1.fastq.gz'):
    read2 = read1.replace('R1', "R2")
    
    size1 = os.stat(read1).st_size
    size2 = os.stat(read2).st_size
    
    if size1 != size2:
#         improperly_paired.append(read1)
        reads.loc[read1, 'properly_paired'] = False
    else:
        reads.loc[read1, 'properly_paired'] = True

/home/obotvinnik/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:810: UserWarning: Found matplotlib configuration in ~/.matplotlib/. To conform with the XDG base directory standard, this configuration location has been deprecated on Linux, and the new location is now /home/obotvinnik/.config/matplotlib/. Please move your configuration there to ensure that matplotlib will continue to find it in the future.
  _get_xdg_config_dir())


In [4]:
from collections import defaultdict

redo_commands = defaultdict(list)

for err in glob.iglob('downsample*err*'):
    filesize = os.stat(err).st_size
    if filesize > 50:
        prefix, number = err.split('-')
        ! cat $err
        sh = prefix.rstrip('.err')
        pattern = '\[{}\]'.format(number)
        lines = ! grep "cmd\[$number\]=" $sh
        line = lines[0]
        command = line.split('=')[1].strip('"')
        redo_commands[sh].append(command)
        split = line.split()
        
        input_fastq = split[5]
        reads = int(split[6])
        output_fastq = split[-1]

=>> PBS: job killed: walltime 925 exceeded limit 900
=>> PBS: job killed: walltime 907 exceeded limit 900
=>> PBS: job killed: walltime 932 exceeded limit 900
=>> PBS: job killed: walltime 939 exceeded limit 900
Can't open /etc/security/access.conf: No such file or directory.
=>> PBS: job killed: walltime 1829 exceeded limit 1800
=>> PBS: job killed: walltime 927 exceeded limit 900
Can't open /etc/security/access.conf: No such file or directory.
=>> PBS: job killed: walltime 943 exceeded limit 900
=>> PBS: job killed: walltime 3627 exceeded limit 3600
ls: cannot access PBS_TEMP_4554889[47].tscc-mgr.local: No such file or directory

real	2m22.582s
user	2m3.107s
sys	0m18.467s
ls: cannot access PBS_TEMP_4554891[5].tscc-mgr.local: No such file or directory

real	1m30.674s
user	1m21.466s
sys	0m9.011s
=>> PBS: job killed: walltime 937 exceeded limit 900
=>> PBS: job killed: walltime 917 exceeded limit 900
=>> PBS: job killed: walltime 937 exceeded limit 900
Can't remove /etc/security/access.

ValueError: too many values to unpack

In [52]:
import pprint

In [56]:
redo_commands.keys()

['downsample75.sh', 'downsample60.sh', 'downsample45.sh', 'downsample100.sh']

In [54]:
pprint.pprint(dict(redo_commands))

{'downsample100.sh': ['time seqtk sample -s 325 /home/obotvinnik/projects/singlecell_pnms/data/P9_01_R2.fastq.gz 80000000 | gzip - > /home/obotvinnik/projects/downsample_singlecell/data/P9_01_80000000reads_iteration2_R2.fastq.gz'],
 'downsample45.sh': ['time seqtk sample -s 146 /home/obotvinnik/projects/singlecell_pnms/data/M6_03_R1.fastq.gz 35000000 | gzip - > /home/obotvinnik/projects/downsample_singlecell/data/M6_03_35000000reads_iteration2_R1.fastq.gz',
                     'time seqtk sample -s 169 /home/obotvinnik/projects/singlecell_pnms/data/P9_01_R2.fastq.gz 40000000 | gzip - > /home/obotvinnik/projects/downsample_singlecell/data/P9_01_40000000reads_iteration4_R2.fastq.gz',
                     'time seqtk sample -s 92 /home/obotvinnik/projects/singlecell_pnms/data/M6_03_R1.fastq.gz 20000000 | gzip - > /home/obotvinnik/projects/downsample_singlecell/data/M6_03_20000000reads_iteration5_R1.fastq.gz',
                     'time seqtk sample -s 89 /home/obotvinnik/projects/singlec

from 1.4.0_downsample_fastq:

    if millions_of_reads == 45:
        sub = qtools.Submitter(commands, 'downsample45', array=True, queue='home-scrm', walltime='00:15:00', ppn=1)
        commands = []
    elif millions_of_reads == 60:
        sub = qtools.Submitter(commands, 'downsample60', array=True, queue='home-scrm', walltime='00:30:00', ppn=2)
        commands = []
    elif millions_of_reads == 75:
        sub = qtools.Submitter(commands, 'downsample75', array=True, queue='home-scrm', walltime='01:00:00', ppn=3)
        commands = []
    elif millions_of_reads == 100:
        sub = qtools.Submitter(commands, 'downsample100', array=True, queue='home-scrm', walltime='02:00:00', ppn=4)
        commands = []

Create new walltimes that are double the time

In [66]:
walltimes = {45: '00:30:00', 60: '01:00:00', 75: '2:00:00', 100:'4:00:00'}
processors = {45:1, 60: 2, 75: 3, 100: 4}

In [67]:
import qtools


for prefix, commands in redo_commands.items():
    jobname = '{}-redo'.format(prefix)
    max_reads = int(prefix.split('downsample')[1].split('.')[0])
    walltime = walltimes[max_reads]
    ppn = processors[max_reads]
    sub = qtools.Submitter(commands, jobname, array=True, queue='home-yeo', walltime=walltime, ppn=ppn)

running 1 tasks as an array-job.
Wrote commands to downsample75.sh-redo.sh
.Submitted script to queue home-yeo. Job ID: 4584343
running 3 tasks as an array-job.
Wrote commands to downsample60.sh-redo.sh
.Submitted script to queue home-yeo. Job ID: 4584344
running 14 tasks as an array-job.
Wrote commands to downsample45.sh-redo.sh
.Submitted script to queue home-yeo. Job ID: 4584345
running 1 tasks as an array-job.
Wrote commands to downsample100.sh-redo.sh
.Submitted script to queue home-yeo. Job ID: 4584346


Re-did downsampling with tiered submission so remove everything that has a single number (tiered submission was with two numbers after "downsample")

In [64]:
! for JOBID in {4584337..4584340} ; do echo $JOBID ; qdel $JOBID[] ; done

4584337
4584338
4584339
4584340


In [45]:
ls -lhaStr downsample*sh

-rw-r--r-- 1 obotvinnik yeo-group 26K Mar 18 13:52 downsample75.sh
-rw-r--r-- 1 obotvinnik yeo-group 21K Mar 18 13:52 downsample60.sh
-rw-r--r-- 1 obotvinnik yeo-group 21K Mar 18 13:52 downsample45.sh
-rw-r--r-- 1 obotvinnik yeo-group 40K Mar 18 13:52 downsample100.sh


In [44]:
! rm -i downsample{1..9}*

rm: remove regular file `downsample1.sh'? ^C

In [38]:
! ls -1 downsample* | grep -E downsample\d | head

^C

In [35]:
! ls -lhaStr downsample4*err*

-rw------- 1 obotvinnik yeo-group  44 Feb 12 16:01 downsample4.sh.err-15
-rw------- 1 obotvinnik yeo-group  44 Feb 12 16:02 downsample4.sh.err-16
-rw------- 1 obotvinnik yeo-group  45 Feb 12 16:02 downsample4.sh.err-19
-rw------- 1 obotvinnik yeo-group  45 Feb 12 16:49 downsample4.sh.err-18
-rw------- 1 obotvinnik yeo-group  46 Feb 12 16:52 downsample4.sh.err-17
-rw------- 1 obotvinnik yeo-group  46 Feb 12 16:54 downsample4.sh.err-35
-rw------- 1 obotvinnik yeo-group  45 Feb 12 17:16 downsample4.sh.err-497
-rw------- 1 obotvinnik yeo-group  44 Feb 12 17:17 downsample4.sh.err-499
-rw------- 1 obotvinnik yeo-group  44 Feb 12 17:18 downsample4.sh.err-498
-rw------- 1 obotvinnik yeo-group  44 Feb 12 17:18 downsample4.sh.err-495
-rw------- 1 obotvinnik yeo-group  43 Feb 12 17:20 downsample4.sh.err-500
-rw------- 1 obotvinnik yeo-group  45 Feb 12 17:21 downsample4.sh.err-496
-rw------- 1 obotvinnik yeo-group  45 Feb 12 17:24 downsample4.sh.err-482
-rw------- 1 obotvinnik yeo-group  44 Feb 12

In [33]:
! ls -lha $err

-rw------- 1 obotvinnik yeo-group 126 Feb 10 03:33 downsample7.sh.err-440


In [32]:
os.stat(err)

posix.stat_result(st_mode=33152, st_ino=576460769986666208, st_dev=3068283210, st_nlink=1, st_uid=132047, st_gid=7198, st_size=126, st_atime=1458751680, st_mtime=1455104015, st_ctime=1455104194)

In [31]:
! cat $err

[E::stk_sample] Could not allocate enough memory for 89000000 sequences. Exiting...

real	0m0.194s
user	0m0.003s
sys	0m0.002s


In [21]:
err

'downsample7.sh.err-440'

In [28]:
! grep 'cmd\[400\]=' $sh

cmd[400]="time seqtk sample -s 2020 /home/obotvinnik/projects/singlecell_pnms/data/M6_03_R2.fastq.gz 104000000 | gzip - > /home/obotvinnik/projects/downsample_singlecell/data/M6_03_104000000reads_iteration9_R2.fastq.gz"


In [ ]:
lines = ! grep 334 downsample1.sh
line = lines[0]
line

['cmd[334]="seqtk',
 'sample',
 '-s',
 '167',
 '/home/obotvinnik/projects/singlecell_pnms/data/P9_01_R2.fastq.gz',
 '12000000',
 '>',
 '/home/obotvinnik/projects/downsample_singlecell/data/P9_01_R2_12000000reads_iteration3.fastq.gz"']

In [2]:
!tail downsample*out*

tail: cannot open `downsample*out*' for reading: No such file or directory


In [34]:
ls -lha $new_folder

lrwxrwxrwx 1 obotvinnik yeo-group 61 Jan 23 08:01 /home/obotvinnik/projects/downsample_singlecell/data -> /home/obotvinnik/scratch/projects/downsample_singlecell/data//
